# Lab 4-1: Data preprocessing. Building a regression model.

In this lab will write a simple regression model to predict the solubility of chemical molecules. We will use the `RDKit` library to calculate molecular descriptors from SMILES strings, and the `scikit-learn` library to build a regression model.

We will learn how to get different molecular decriptors from SMILES strings with the use od `RDKit` library. Then we will preprocess data, implement a featurizer for automatic input data preparation, and wrap the whole process in a simple, user-friendly web app using `Streamlit`.

---



In [5]:
import pandas as pd

df = pd.read_csv('data/aqsol.csv')
df.head()

,Name,SMILES,Solubility
0,"N,N,N-trimethyloctadecan-1-aminium bromide",[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127
1,Benzo[cd]indol-2(1H)-one,O=C1Nc2cccc3cccc1c23,-3.254767
2,4-chlorobenzaldehyde,Clc1ccc(C=O)cc1,-2.177078
3,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409
4,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065


## Calculating molecular descriptors with RDKit

**RDKit is a collection of cheminformatics tools, widely used to analyze and process chemical data.** We will use some of its functionalities to calculate molecular descriptors for the molecules in our dataset, and use the descriptors as input features for a regression model that will learn to predict the solubility of the molecules from SMILES strings. 

For this task will use only one submodule of RDKit, `rdkit.Chem.rdMolDescriptors`, which contains functions for calculating the molecular descriptors we are interested in. Feel free to explore RDKit yourself by reading [rdMolDescriptors documentation](https://www.rdkit.org/docs/source/rdkit.Chem.rdMolDescriptors.html) and [RDKit documentation](https://www.rdkit.org/docs/index.html).

The most important RDKit class is `Mol` which represents a molecule with its atoms, bonds, spatial conformation, etc. Most RDKit functions, including those for calculating molecular descriptors, take a `Mol` object as input. If you have a SMILES string, you can create a `Mol` object using the `Chem.MolFromSmiles` function, as shown below:

```python
from rdkit import Chem

smiles = 'C1C(=O)NC2=C(C=C(C=C2)[N+](=O)[O-])C(=N1)C3=CC=CC=C3'
mol = Chem.MolFromSmiles(smiles)
```

In [6]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit import RDLogger  
RDLogger.DisableLog('rdApp.*') # Disabling rdkit warnings

# First, we will create a mol object for each molecule in the dataset, and store it in a new column
df['mol'] = df['SMILES'].apply(Chem.MolFromSmiles)

# Now we can calculate the molecular descriptors
df['mol_wt'] = df['mol'].apply(rdMolDescriptors.CalcExactMolWt)             # Molecular weight
df[['logp', 'mr']] = df['mol'].apply(rdMolDescriptors.CalcCrippenDescriptors).apply(pd.Series)  # LogP (lipophilicity)
df['num_heavy_atoms'] = df['mol'].apply(rdMolDescriptors.CalcNumHeavyAtoms) # Number of heavy atoms
df['num_HBD'] = df['mol'].apply(rdMolDescriptors.CalcNumHBD)                # Number of hydrogen bond donors
df['num_HBA'] = df['mol'].apply(rdMolDescriptors.CalcNumHBA)                # Number of hydrogen bond acceptors
df['aromatic_rings'] = df['mol'].apply(rdMolDescriptors.CalcNumAromaticRings) # Number of aromatic rings

df.head()

,Name,SMILES,Solubility,mol,mol_wt,logp,mr,num_heavy_atoms,num_HBD,num_HBA,aromatic_rings
0,"N,N,N-trimethyloctadecan-1-aminium bromide",[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,<rdkit.Chem.rdchem.Mol object at 0x000001DF7B5...,391.281363,3.9581,102.4454,23,0,0,0
1,Benzo[cd]indol-2(1H)-one,O=C1Nc2cccc3cccc1c23,-3.254767,<rdkit.Chem.rdchem.Mol object at 0x000001DF7B5...,169.052764,2.4055,51.9012,13,1,1,2
2,4-chlorobenzaldehyde,Clc1ccc(C=O)cc1,-2.177078,<rdkit.Chem.rdchem.Mol object at 0x000001DF7B5...,140.002892,2.1525,36.8395,9,0,1,1
3,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,<rdkit.Chem.rdchem.Mol object at 0x000001DF7B5...,754.227281,8.1161,200.7106,53,2,6,6
4,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,<rdkit.Chem.rdchem.Mol object at 0x000001DF7B5...,422.220557,2.4854,119.0760,31,0,6,2


## Exercise 1: Extract features and split the dataset (1 point)

Now that we have the molecular descriptors, we can use them as input features for a regression model. As we did in the previous lab, we will extract the **input features** $X$ and the target variable $y$, and split the dataset into training and test sets.

Let's use the newly calculated molecular descriptors as input features, and the solubility as the target variable.

1. Extract the input features and the target variable from the dataset
2. Split the dataset into training and test sets, with a test size of 20%

In [37]:
# Select only the relevant columns of the dataframe (features and target)

y = df['Solubility']
X = df[['mol_wt', 'num_heavy_atoms', 'num_HBD', 'num_HBA', 'aromatic_rings', 'logp']]

In [38]:
# Split the dataset into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


## Exercise 2: Build a regression model (2 points)

You already know the `scikit-learn` library, as we used it to build some classifier models in the previous labs. Now, we will use it to build a regression model. Linear regression is the simplest regression model, and it is a good starting point for regression problems. It is implemented in scikit-learn as `LinearRegression`. You should also try a more complex model, such as `SVR` (Support Vector Regression) and compare the results.

1. Train a `LinearRegression` model on the training set. Report $RMSE$ (Root Mean Squared Error) and $R^2$ score on the train and test sets.
2. Train an `SVR` model on the training set. Report $RMSE$ and $R^2$ score on the train and test sets. **Use the default hyperparameters for now.**

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import math 

lr = LinearRegression()
lr.fit(X_train, y_train)

train_pred_lr = lr.predict(X_train)  # make predictions on the training set
test_pred_lr = lr.predict(X_test)  # make predictions on the testing set

RMSE_lr_train = math.sqrt(mean_squared_error(y_train, train_pred_lr))
R_2_lr_train = mean_squared_error(y_train, train_pred_lr)
print('Training RMSE for LR:', RMSE_lr_train)
print('Training R^2 for LR:', R_2_lr_train)

RMSE_lr_test = math.sqrt(mean_squared_error(y_test, test_pred_lr))
R_2_lr_test = mean_squared_error(y_test, test_pred_lr)
print('Testing RMSE for LR:', RMSE_lr_test)
print('Testing R^2 for LR:', R_2_lr_test)



Training RMSE for LR: 1.7547750884915405
Training R^2 for LR: 3.079235611190494
Testing RMSE for LR: 1.6817143596288688
Testing R^2 for LR: 2.828163187381936


In [10]:
svr = SVR()
svr.fit(X_train, y_train)

train_pred_svr = svr.predict(X_train)  # make predictions on the training set
test_pred_svr = svr.predict(X_test)  # make predictions on the testing set

RMSE_svr_train = math.sqrt(mean_squared_error(y_train, train_pred_svr))
R_2_svr_train = mean_squared_error(y_train, train_pred_svr)
print('Training RMSE for SVR:', RMSE_svr_train)
print('Training R^2 for SVR:', R_2_svr_train)

RMSE_svr_test = math.sqrt(mean_squared_error(y_test, test_pred_svr))
R_2_svr_test = mean_squared_error(y_test, test_pred_svr)
print('Testing RMSE for SVR:', RMSE_svr_test)
print('Testing R^2 for SVR:', R_2_svr_test)

Training RMSE for SVR: 1.7184323912412482
Training R^2 for SVR: 2.9530098832671143
Testing RMSE for SVR: 1.7244337800149632
Testing R^2 for SVR: 2.9736718616566944


## Preprocessing data

**Standardization** of datasets is a common requirement for many machine learning estimators implemented in scikit-learn. If some features of our data have very different scales (for example, one feature is in the range $[0, 1]$ and another can potentially be any positive number), **some models** might consider the feature with larger numerical values to be more important. This can be a problem, as we want our model to be able to learn from all features equally.

Standardization transform our data in such a way that its distribution will have a mean value $\mu = 0$ and standard deviation $\sigma = 1$. We can achieve this by using the `StandardScaler` from the `sklearn` library.

Another common preprocessing step is **normalization**. In this case, the data is scaled to a fixed range, usually $[0, 1]$. The motivation to use this scaling method includes preserving zeros in sparse data. For example, when making predictions about the expected outcome of an anticancer therapy, the value of $0$ observed tumors in a patient is probably much more informative for the predictive model than any other number of observed tumors alone. We can scale data to a certain range by using the `MinMaxScaler` from the `sklearn` library.

You can read more about those two feature scaling methods in the [Scikit-learn preprocessing docs](https://scikit-learn.org/stable/modules/preprocessing.html).

<center>
    <img src="imgs/logp-std.png" width="500">
</center>
<br>

**Remember that the scaler should be fitted only on the training set, and then used to transform both the training and test sets.** If we first transform the entire dataset and then split it into training and test sets, we are leaking the information about **test set data distribution** to the model, which would lead to overly optimistic results.

```python
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

...
```


## Exercise 3: Preprocess the input features (2 points)

1. Standardize the input features using the `StandardScaler` from `sklearn`. **Fit the scaler on the training set and then transform both the training and test sets**.
2. Train both linear regression and SVR models on the standardized training set. Report $RMSE$ and $R^2$ score on the train and test sets. How do the results compare to the previous models? Which model benefits from standardization?

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_st = scaler.fit_transform(X_train)
X_test_st = scaler.transform(X_test)


In [12]:
lr.fit(X_train_st, y_train)

train_pred_lr_st = lr.predict(X_train_st)  # make predictions on the standardized training set
test_pred_lr_st = lr.predict(X_test_st)  # make predictions on the standardized testing set

RMSE_lr_train_st = math.sqrt(mean_squared_error(y_train, train_pred_lr_st))
R_2_lr_train_st = mean_squared_error(y_train, train_pred_lr_st)
print('Training RMSE for LR:', RMSE_lr_train_st)
print('Training R^2 for LR:', R_2_lr_train_st, '\n')

RMSE_lr_test_st = math.sqrt(mean_squared_error(y_test, test_pred_lr_st))
R_2_lr_test_st = mean_squared_error(y_test, test_pred_lr_st)
print('Testing RMSE for LR:', RMSE_lr_test_st)
print('Testing R^2 for LR:', R_2_lr_test_st)



Training RMSE for LR: 1.7262282971237677
Training R^2 for LR: 2.9798641337908225 

Testing RMSE for LR: 1.7985230501194502
Testing R^2 for LR: 3.23468516181097


In [13]:
svr.fit(X_train_st, y_train)

train_pred_svr_st = svr.predict(X_train_st)  # make predictions on the standardized training set
test_pred_svr_st = svr.predict(X_test_st)  # make predictions on the standardized testing set

RMSE_svr_train_st = math.sqrt(mean_squared_error(y_train, train_pred_svr_st))
R_2_svr_train_st = mean_squared_error(y_train, train_pred_svr_st)
print('Training RMSE for SVR:', RMSE_svr_train_st)
print('Training R^2 for SVR:', R_2_svr_train_st, '\n')

RMSE_svr_test_st = math.sqrt(mean_squared_error(y_test, test_pred_svr_st))
R_2_svr_test_st = mean_squared_error(y_test, test_pred_svr_st)
print('Testing RMSE for SVR:', RMSE_svr_test_st)
print('Testing R^2 for SVR:', R_2_svr_test_st)

Training RMSE for SVR: 1.2838120846083922
Training R^2 for SVR: 1.648173468586546 

Testing RMSE for SVR: 1.3651564538888732
Testing R^2 for SVR: 1.8636521435944429


## *Exercise: Featurizer class

All the data manipulations, including the generation of features (molecular descriptors) and scaling we did completely by hand. Imagine that we have a new dataset with molecules for which we want to predict the solubility. We would have to repeat all the steps we did above, which is not at all user-friendly.

Let's encapsulate all the steps in a single class called `Featurizer`. 

**Featurizer is a piece of code which takes care of extracting the features and preparing them** in such a way that they can be used as an input for a ML model. In this fashion, our featurizer will be able to take a list of SMILES strings and return a dataframe with the molecular descriptors, ready to be used in any regression model.

The `Featurizer` class should have the following methods:

1. `__init__(self, smiles)`

    The constructor method should take an array of **training set** SMILES strings as input and calculate the molecular descriptors for each molecule (in a `pandas.DataFrame`, as we did earlier). **It should also fit a scaler to the descriptors.** The descriptors should be stored in a dataframe, and the scaler should be stored as an attribute of the class.
      

2. `featurize(self, smiles)`
    
   This method should take an array of SMILES strings as input and return a dataframe with the molecular descriptors calculated with RDKit. **The descriptors should be standardized using the scaler which was fitted to the training set in the `__init__` method**.

3. **You are more than welcome to implement other methods** if you think it will make your code cleaner or more efficient. Maybe we could have a separate method for the calculation of molecular descriptors, or for the scaling of the features?


### Here is an example of how our `Featurizer` class should be used
```python
featurizer = Featurizer(X_train['SMILES'])  # Create an instance of the Featurizer class, passing the training set SMILES strings

# Say we want to predict the solubility of the following molecules
some_SMILES = ['CC(=O)Oc1ccccc1C(=O)O',     
               'C1=CC(=C(C=C1[C@H](CN)O)O)O',
               'C1=NC2=C(C(=N1)N)N=CN2C3C(C(C(O3)CO)O)O']

# Extract the features
X = featurizer.featurize(some_SMILES)

...

y_pred = model.predict(X)   # Get predictions
```


In [31]:
class Featurizer:
    import pandas as pd
    scaler_ft = StandardScaler()

    def __init__(self, smiles):
        self.smiles = self['SMILES']
        self.data = pd.DataFrame({'SMILES': smiles})
        self.data['mol'] = self.data['SMILES'].apply(Chem.MolFromSmiles)
        # Calculate all the molecular descriptors
        self.data['mol_wt'] = self.data['mol'].apply(rdMolDescriptors.CalcExactMolWt)             # Molecular weight
        self.data[['logp', 'mr']] = self.data['mol'].apply(rdMolDescriptors.CalcCrippenDescriptors).apply(pd.Series)  # LogP (lipophilicity)
        self.data['num_heavy_atoms'] = self.data['mol'].apply(rdMolDescriptors.CalcNumHeavyAtoms) # Number of heavy atoms
        self.data['num_HBD'] = self.data['mol'].apply(rdMolDescriptors.CalcNumHBD)                # Number of hydrogen bond donors
        self.data['num_HBA'] = self.data['mol'].apply(rdMolDescriptors.CalcNumHBA)                # Number of hydrogen bond acceptors
        self.data['aromatic_rings'] = self.data['mol'].apply(rdMolDescriptors.CalcNumAromaticRings) # Number of aromatic rings

        #Scaling part:
        descriptor_columns = ['mol_wt', 'num_heavy_atoms', 'num_HBD', 'num_HBA', 'aromatic_rings', 'logp']
        self.descriptor_data = self.data[descriptor_columns]
        self.scaler_ft.fit(self.descriptor_data)


    def featurize(self, smiles):
        self.smiles = smiles
        self.new_data = pd.DataFrame({'SMILES': smiles})
        self.new_data['mol'] = self.new_data['SMILES'].apply(Chem.MolFromSmiles)
        
        # Calculate all the molecular descriptors
        self.new_data['mol_wt'] = self.new_data['mol'].apply(rdMolDescriptors.CalcExactMolWt)             # Molecular weight
        self.new_data[['logp', 'mr']] = self.new_data['mol'].apply(rdMolDescriptors.CalcCrippenDescriptors).apply(pd.Series)  # LogP (lipophilicity)
        self.new_data['num_heavy_atoms'] = self.new_data['mol'].apply(rdMolDescriptors.CalcNumHeavyAtoms) # Number of heavy atoms
        self.new_data['num_HBD'] = self.new_data['mol'].apply(rdMolDescriptors.CalcNumHBD)                # Number of hydrogen bond donors
        self.new_data['num_HBA'] = self.new_data['mol'].apply(rdMolDescriptors.CalcNumHBA)                # Number of hydrogen bond acceptors
        self.new_data['aromatic_rings'] = self.new_data['mol'].apply(rdMolDescriptors.CalcNumAromaticRings) # Number of aromatic rings

        #Scaling part:
        descriptor_columns = ['mol_wt', 'num_heavy_atoms', 'num_HBD', 'num_HBA', 'aromatic_rings', 'logp']
        descriptors = self.new_data[descriptor_columns]
        st_descriptors = self.scaler_ft.transform(descriptors)
        st_descriptors = pd.DataFrame(descriptors, columns = descriptor_columns)
        return st_descriptors
        

In [32]:
# Test your featurizer

featurizer = Featurizer(X_train['SMILES']) # Pass the training set SMILES strings to the constructor

import pandas as pd

tryptamines = pd.read_csv('data/tryptamines.csv')   # Load a new dataset
tryptamines.head()

KeyError: 'SMILES'

In [2]:
X = featurizer.featurize(tryptamines['SMILES'])   # Extract features for the new dataset
X.head()

NameError: name 'featurizer' is not defined

In [ ]:
# Predict the solubility of the new molecules

y_pred = svr.predict(X)  # Use the linear regression model to predict the solubility

tryptamines['Solubility'] = y_pred  # Add the predictions to the dataframe
tryptamines.head()

## Cross-validation and grid search

Now that we have a working regression model, we can try to improve its performance by tuning the hyperparameters. Last time we tuned the hyperparameters by hand, but as you may guess, that is not the most efficient way to do it. What we can (and will) do is employ a **search algorithm** to find the best hyperparameters automatically. This piece of code will try different hyperparameters and return the best combination.

Last time we introduced the concept of a **validation set**, which is a subset of the training set used to evaluate the model's performance during hyperparameter tuning. To tune the hyperparameters of a classifier model we cut out a part of the training set and used the accuracy on this validation set as a metric to evaluate the model's performance.

In practice, when tuning hyperparameters of a machine learning model, we do not usually create a single validation set. Instead, we employ a srategy called **cross-validation**. In $k$-fold cross-validation, the process of training and evaluating the model is repeated $k$ times, with each repetition using a different validation set. Thanks to this strategy, each data point present in the original training set will have the chance to be included in a validation sets. The metrics reported from $k$ folds are then averaged to get a more reliable estimate of the model's performance.

<center>
<img src="imgs/cross-validation.png" width="800">
</center>

Luckily, we do not have to implement this algorithm ourselves, as scikit-learn provides a `GridSearchCV` class that can help us with this task. It performs an exhaustive search over a specified parameter grid and evaluates the model's performance using $k$-fold cross-validation. By default, `GridSearchCV` uses a 5-fold cross-validation, which is a common choice for $k$.

In [41]:
from sklearn.model_selection import GridSearchCV

# We will define a grid of hyperparameters as a dictionary. The keys are the hyperparameter names, and the values are lists of possible values to try.

param_grid = {
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5]
}

grid_search = GridSearchCV(
    svr, # the model
    param_grid, # the grid of hyperparameters
    verbose=2 # print the progress
)

svr = grid_search.fit(X_train, y_train) # GridSearchCV.fit() returns the best model, and we can save it to a new variable

Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV] END .......................C=1, max_iter=2, shrinking=2; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=2; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=2; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=2; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=2; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=4; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=4; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=4; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=4; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=4; total time=   0.0s


K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_valida

[CV] END .......................C=1, max_iter=2, shrinking=6; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=6; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=6; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=6; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=6; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=8; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=8; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=8; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=8; total time=   0.0s
[CV] END .......................C=1, max_iter=2, shrinking=8; total time=   0.0s
[CV] END ......................C=1, max_iter=2, shrinking=10; total time=   0.0s
[CV] END ......................C=1, max_iter=2, shrinking=10; total time=   0.0s
[CV] END ...................

K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_valida

[CV] END .......................C=1, max_iter=4, shrinking=2; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=2; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=2; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=4; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=4; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=4; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=4; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=4; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=6; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=6; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=6; total time=   0.0s
[CV] END .......................C=1, max_iter=4, shrinking=6; total time=   0.0s
[CV] END ...................

...

[CV] END .......................C=5, max_iter=8, shrinking=2; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=4; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=4; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=4; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=4; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=4; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=6; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=6; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=6; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=6; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=6; total time=   0.0s
[CV] END .......................C=5, max_iter=8, shrinking=8; total time=   0.0s
[CV] END ...................

K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_valida

[CV] END ......................C=5, max_iter=10, shrinking=2; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=2; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=2; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=2; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=4; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=4; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=4; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=4; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=4; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=6; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=6; total time=   0.0s
[CV] END ......................C=5, max_iter=10, shrinking=6; total time=   0.0s
[CV] END ...................

K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


ValueError: 
All the 625 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
625 fits failed with the following error:
Traceback (most recent call last):
  File "K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\svm\_base.py", line 252, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "K:\Gry\anaconda\envs\pum24\lib\site-packages\sklearn\svm\_base.py", line 331, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx", line 191, in sklearn.svm._libsvm.fit
ValueError: shrinking != 0 and shrinking != 1


In [ ]:
from sklearn.metrics import mean_squared_error

# Report the best hyperparameters and RMSE on the testing set

print('Best hyperparameters:', svr.get_params())

y_pred = svr.predict(X_test)

print('-'*50)
print('Testing set rmse:', mean_squared_error(y_test, y_pred))

### *Randomized search

`RandomizedSearchCV` is an alternative to `GridSearchCV`. Instead of trying all possible combinations of fixed hyperparameters, it samples a fixed number of hyperparameter settings from specified probability distributions. Although you may be hesitant to use it, as it is not an exhaustive search, it is often more efficient than grid search. Randomized search is especially useful when we have many hyperparameters to tune, and we are not sure which ones are the most important. Take a look at the figure below and try to understand why this is the case.

<center>
<img src="imgs/grid-vs-random.png", width="600">
</center>

Randomized search is implemented in the same way as grid search, but with a different class. It also requires a dictionary of hyperparameters, but instead of specifying a list of values to try, we specify a **probability distribution**. See the example below:

```python
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_distributions = {
    'some_discete_hyperparameter': np.arange(1, 10), # uniform dicrete distribution from 1 to 10
    'some_continuous_hyperparameter': uniform(0, 1), # uniform continuous distribution from 0 to 1
}

random_search = RandomizedSearchCV(
    model,
    param_distributions,
    n_iter=100, # number of random samples to try

...
```

## Saving and loading Python objects

Both the featurizer and the trained regression model can be saved to disk as Python objects using the `pickle` module. This way, we can load them later in another script without the need to retrain the model or recalculate the molecular descriptors.

```python
import pickle

# Save the model

with open('path/to/model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Load the model

with open('path/to/model.pkl', 'rb') as f:
    model = pickle.load(f)

---
# Lab 4-2: Coding a simple app in Streamlit.

Streamlit is a Python library that allows you to create very nice-looking and user-friendly web applications with just a few lines of code. It is very easy to use and requires absolutely no knowledge of HTML, CSS, or JavaScript.

---

We should have Streamlit already installed in our environment. You probably should take a look at the [Streamlit documentation](https://docs.streamlit.io/library), but this is how you can run a simple Streamlit app:

Take look at `data/streamlit/app.py`. It is a tiny Streamlit app that takes a name as input and **returns a random fortune cookie-like quote**. Analyze the code and try to understand how it works. 

**Some important functions you may need to use:**

- `st.title` sets the title of the app
- `st.write` writes text to the app (supports [Markdown](https://www.markdownguide.org/basic-syntax/) synthax for text formatting)
- `st.dataframe` displays a dataframe
- `st.latex` renders LaTeX code (for mathematical formulas)
- `st.text_input` creates a text input field (for multiline inputs, use `st.text_area`)
- `st.button` creates a button (you can use it to trigger some action)

To run the app, open a terminal and run the following command:

    streamlit run data/streamlit/app.py
    
This will start a local server and open a new tab in your browser with the app. You can now interact with the app by entering your name and clicking the button to get a random fortune.

## Exercise 5: Build an app for solubility prediction (3 points)

This exercise is a continuation of the previous one. We will use all the code we have written so far to build a simple web app that predicts the solubility of molecules. The app will run in the browser and allow the user to input one or more SMILES strings, and it will display the predicted solubility of the molecules.
The [Streamlit documentation](https://docs.streamlit.io/library) will be very helpful in this task.

**The app should have the following structure:**

1. A title and a short description of what the app does.
2. A text area where the user can input a SMILES string (or multiple SMILES strings in a column).
3. A button to submit the input.
4. A section that displays the predicted solubility of the molecule(s) as a table of SMILES strings and their corresponding solubility values.

The app should use our `Featurizer` class to extract the features from the input SMILES strings and predict the solubility using the trained model. **The model and the featurizer should not be trained inside the app, but loaded from a `.pkl` file instead!** Thanks to this approach, we will not need the training data to run the app. Both the model and the featurizer are already trained and saved as Python objects.

In [ ]:
# For this exercise, you should create a new Python script called in the 'data/streamlit' directory and implement the app there.